In [ ]:
import seaborn as sns
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "drive/MyDrive/Quora-QnA/quora-question-pairs" -d "quora-question-pairs"
!unzip "quora-question-pairs/train.csv.zip"
!unzip "quora-question-pairs/test.csv.zip"

In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
df = df.iloc[:100]

In [ ]:
len(df)

In [ ]:
df.head(3)

In [ ]:
df.groupby("is_duplicate")['id'].count().plot.bar()

Nearly 37% of the rows provided are for duplicate pair examples

---



In [ ]:
print("Total unique questions: " , (df['qid1'].nunique()+df['qid2'].nunique()))

In [ ]:
for i in df['qid1']:
  num = int(i)+1

In [ ]:
for i in df['qid1']:
  num = int(i)+1

The above 2 cells did not return an error. It means that all the question ids in column are numbers as expected

In [ ]:
map = {}
for i in range(len(df)):
  if i%50000 == 0:
    print(i)
  x = str(df.iloc[i]['qid1'])+str(df.iloc[i]['qid2'])
  if (x in map.keys()):
    print(x, " ==> duplicate")
  map[x] = 1

No duplicate question pair is present in the dataset

###### Filling null values in dataset with ":"

In [ ]:
nan_rows = df[df.isnull().any(1)]
df = df.fillna('')
nan_rows = df[df.isnull().any(1)]

##### Checking for how many question are lss than or equal to 2 words

In [ ]:
counter = 0
ls =[]
to_remove_indices = []
for i in df['question1']:
  i = str(i).split()
  if len(i) <= 2:
    ls.append(counter)
  counter+=1

In [ ]:
len(df)

In [ ]:
for i in range(len(ls)-1,-1,-1):
  df = df.drop(ls[i])

In [ ]:
len(df)

140 rows removed where one of the questions was less than 2 words of length

### Basic Feature Extraction

##### Extracting feature: common words and ratio of common words to total words in question

We will creae new columns in the table as following:


> 1. w_q1: number of words in q1
> 2. w_q2: number of words in q2
> 3. w_common: number of common words in q1 and q2
> 4. ratio_q1 = w_common/w_q1
> 5. ratio_q1 = w_common/w_q2




In [ ]:
ls1 = df['question1']
ls2 = df['question2']

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
tokenized_q1 = []
tokenized_q2 = []

In [ ]:
for q in ls1:
  tokenized_q1.append(nlp(q))
for q in ls2:
  tokenized_q2.append(nlp(q))

In [ ]:
df['w_q1'] = pd.Series(tokenized_q1).apply(lambda ar :len(ar))
df['w_q2'] = pd.Series(tokenized_q2).apply(lambda ar :len(ar))

In [ ]:
common_words = []
for i in range(len(df)):
  map = {}
  count= 0
  for token in tokenized_q1[i]:
    map[token.text] = True
  for token in tokenized_q2[i]:
    if token.text in map.keys():
      count+=1
  common_words.append(count)

df['w_comm'] = common_words

In [ ]:
df['ratio_q1'] = df['w_comm']/df['w_q1']
df['ratio_q2'] = df['w_comm']/df['w_q2']
df['jacc'] = df['w_comm']/(df['w_q1']+df['w_q2'])

In [ ]:
df

In [ ]:
df.iloc[79]

In [ ]:
temp1 = df[df['ratio_q1'] > 1]['id']
temp2 = df[df['ratio_q1'] > 1]['id']
temp3 = []
for i in temp1:
  temp3.append(i)
for i in temp2:
  temp3.append(i)
temp3.sort()
for i in range(len(temp3)-1,0,-1):
  try:
    df = df.drop(temp3[i])
  except:
    ;
del temp1, temp2, temp3

In [ ]:
# sns.displot(df[df['is_duplicate'] == 1]['ratio_q1'], bins = 20)
sns.displot(df[df['is_duplicate'] == 1]['jacc'], bins = 20)

Majority of the questions have 80% of the words same. 
An unexpected spike at 50% is also observed.

##### Taking a look at a feature :

In [ ]:
# plot graph to check ratio only for the smaller(larger) question in size

In [ ]:
1. Unigram distribution comput counts 
2. Try to compute KL divergence for questions
3. do sample analysis